In [1]:
import json
import os
import pandas as pd
import zipfile

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
input_folder = "privacy_scraper/blacklight_json"

# Initialize a list to store rows for the DataFrame
rows = []

# Iterate over each JSON file in the folder
for filename in os.listdir(input_folder):
    if filename.endswith(".json"):
        file_path = os.path.join(input_folder, filename)
        
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
            
            domain_name = filename.replace(".json", "")
            cards = data.get("groups", [])[0].get("cards", [])
            
            ddg_join_ads = 0
            third_party_cookies = 0
            canvas_fingerprinting = 0
            session_recording = 0
            key_logging = 0
            fb_pixel = 0
            google_analytics = 0

            for card in cards:
                card_type = card.get("cardType", "")
                big_number = card.get("bigNumber", 0)

                if card_type == "ddg_join_ads":
                    ddg_join_ads = big_number
                elif card_type == "cookies":
                    third_party_cookies = big_number
                elif card_type == "canvas_fingerprinters":
                    canvas_fingerprinting = 1 if card.get("testEventsFound", False) else 0
                elif card_type == "session_recorders":
                    session_recording = 1 if card.get("testEventsFound", False) else 0
                elif card_type == "key_logging":
                    key_logging = 1 if card.get("testEventsFound", False) else 0
                elif card_type == "fb_pixel_events":
                    fb_pixel = 1 if card.get("testEventsFound", False) else 0
                elif card_type == "ga":
                    google_analytics = 1 if card.get("testEventsFound", False) else 0
            
            rows.append({
                "filename": domain_name,
                "ddg_join_ads": ddg_join_ads,
                "third_party_cookies": third_party_cookies,
                "canvas_fingerprinting": canvas_fingerprinting,
                "session_recording": session_recording,
                "key_logging": key_logging,
                "fb_pixel": fb_pixel,
                "google_analytics": google_analytics,
            })
        
        except Exception as e:
            print(f"Error processing {filename}: {e}")

In [5]:
df = pd.DataFrame(rows)
df.head()

,filename,ddg_join_ads,third_party_cookies,canvas_fingerprinting,session_recording,key_logging,fb_pixel,google_analytics
0,www.freienbach.ch_,0,0,0,0,1,0,0
1,drangsnes.is_,0,0,0,0,0,0,0
2,www.wald.zh.ch_,0,0,0,0,1,0,0
3,www.bournens.ch_,1,0,0,0,0,0,0
4,hagneck.ch_,3,0,0,0,0,0,0


In [6]:
df.to_csv("../data/blacklight_domain.csv", index = False)

In [7]:
df.describe()

,ddg_join_ads,third_party_cookies,canvas_fingerprinting,session_recording,key_logging,fb_pixel,google_analytics
count,3710.000000,3710.000000,3710.000000,3710.000000,3710.000000,3710.000000,3710.000000
mean,0.379515,0.201078,0.003774,0.004582,0.084906,0.007008,0.000809
std,0.847123,0.980532,0.061322,0.067546,0.278779,0.083432,0.028429
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8.000000,15.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
govdir = pd.read_csv("../data/combined_tsv_data.tsv", sep = "\t")
govdir.head()

,Name,Govdirectory URL,Type,Website,source_file
0,Cherkasy Oblast,https://www.govdirectory.org/ukraine/Q161808/,oblast of Ukraine,https://www.oblradack.gov.ua/,Ukraine
1,Chernihiv Oblast,https://www.govdirectory.org/ukraine/Q167874/,oblast of Ukraine,https://cg.gov.ua/,Ukraine
2,Chernivtsi Oblast,https://www.govdirectory.org/ukraine/Q168856/,oblast of Ukraine,https://bukoda.gov.ua/,Ukraine
3,Dnipropetrovsk Oblast,https://www.govdirectory.org/ukraine/Q170672/,oblast of Ukraine,http://www.adm.dp.gov.ua/,Ukraine
4,Donetsk Oblast,https://www.govdirectory.org/ukraine/Q2012050/,oblast of Ukraine,https://dn.gov.ua/,Ukraine


In [9]:
# Keys
df['filename'] = df['filename'].str.rstrip('_')
govdir['Website'] = govdir['Website'].str.replace(r'^https?://', '', regex=True)
govdir['Website'] = govdir['Website'].str.rstrip('/')

In [10]:
fin_df = govdir.merge(df, 
            left_on = "Website",
            right_on = "filename",
                     how = "left")

In [11]:
fin_df.shape

(12255, 13)

In [12]:
fin_df.head()

,Name,Govdirectory URL,Type,Website,source_file,filename,ddg_join_ads,third_party_cookies,canvas_fingerprinting,session_recording,key_logging,fb_pixel,google_analytics
0,Cherkasy Oblast,https://www.govdirectory.org/ukraine/Q161808/,oblast of Ukraine,www.oblradack.gov.ua,Ukraine,www.oblradack.gov.ua,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Chernihiv Oblast,https://www.govdirectory.org/ukraine/Q167874/,oblast of Ukraine,cg.gov.ua,Ukraine,cg.gov.ua,4.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Chernivtsi Oblast,https://www.govdirectory.org/ukraine/Q168856/,oblast of Ukraine,bukoda.gov.ua,Ukraine,bukoda.gov.ua,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Dnipropetrovsk Oblast,https://www.govdirectory.org/ukraine/Q170672/,oblast of Ukraine,www.adm.dp.gov.ua,Ukraine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Donetsk Oblast,https://www.govdirectory.org/ukraine/Q2012050/,oblast of Ukraine,dn.gov.ua,Ukraine,dn.gov.ua,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
fin_df.groupby('source_file').describe().reset_index().reset_index().dropna()

index  source_file ddg_join_ads                                           \
                             count      mean       std  min  25%  50%   75%   
0      0      Austria        389.0  0.154242  0.539056  0.0  0.0  0.0  0.00   
1      1      Belgium        444.0  0.423423  0.931511  0.0  0.0  0.0  1.00   
2      2      Bermuda          2.0  3.000000  1.414214  2.0  2.5  3.0  3.50   
3      3     Cameroon         12.0  0.250000  0.452267  0.0  0.0  0.0  0.25   
7      7   East Timor          6.0  0.000000  0.000000  0.0  0.0  0.0  0.00   
10    10        Ghana         50.0  0.700000  1.388730  0.0  0.0  0.0  1.00   
11    11      Iceland         59.0  1.016949  1.370773  0.0  0.0  1.0  1.00   
15    15       Latvia         18.0  0.333333  0.685994  0.0  0.0  0.0  0.00   
17    17     Malaysia         29.0  0.896552  1.113066  0.0  0.0  1.0  1.00   
20    20        Nepal         81.0  1.703704  1.452966  0.0  0.0  1.0  3.00   
27    27       Russia          8.0  0.500000  0.534522  0.0  0.0  0.5  1.00   
30    30       Sweden        427.0  0.210773  0.655476  0.0  0.0  0.0  0.00   
31    31  Switzerland       1941.0  0.325090  0.748837  0.0  0.0  0.0  0.00   
32    32      Ukraine         27.0  1.481481  1.451004  0.0  1.0  1.0  2.00   

        third_party_cookies                                                \
    max               count      mean       std  min  25%  50%  75%   max   
0   4.0               389.0  0.169666  0.980219  0.0  0.0  0.0  0.0  14.0   
1   7.0               444.0  0.189189  1.085621  0.0  0.0  0.0  0.0  12.0   
2   4.0                 2.0  0.000000  0.000000  0.0  0.0  0.0  0.0   0.0   
3   1.0                12.0  0.333333  1.154701  0.0  0.0  0.0  0.0   4.0   
7   0.0                 6.0  0.000000  0.000000  0.0  0.0  0.0  0.0   0.0   
10  6.0                50.0  0.340000  1.623928  0.0  0.0  0.0  0.0  11.0   
11  5.0                59.0  0.406780  1.131350  0.0  0.0  0.0  0.0   5.0   
15  2.0                18.0  0.333333  0.485071  0.0  0.0  0.0  1.0   1.0   
17  4.0                29.0  0.724138  1.810615  0.0  0.0  0.0  0.0   8.0   
20  4.0                81.0  0.481481  1.817355  0.0  0.0  0.0  0.0   8.0   
27  1.0                 8.0  0.750000  1.388730  0.0  0.0  0.0  1.0   4.0   
30  7.0               427.0  0.213115  1.018153  0.0  0.0  0.0  0.0  15.0   
31  8.0              1941.0  0.164348  0.847243  0.0  0.0  0.0  0.0  11.0   
32  5.0                27.0  0.444444  1.339728  0.0  0.0  0.0  0.0   5.0   

   canvas_fingerprinting                                               \
                   count      mean       std  min  25%  50%  75%  max   
0                  389.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
1                  444.0  0.009009  0.094594  0.0  0.0  0.0  0.0  1.0   
2                    2.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
3                   12.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
7                    6.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
10                  50.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
11                  59.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
15                  18.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
17                  29.0  0.034483  0.185695  0.0  0.0  0.0  0.0  1.0   
20                  81.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
27                   8.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   
30                 427.0  0.007026  0.083623  0.0  0.0  0.0  0.0  1.0   
31                1941.0  0.002576  0.050702  0.0  0.0  0.0  0.0  1.0   
32                  27.0  0.037037  0.192450  0.0  0.0  0.0  0.0  1.0   

   session_recording                                              key_logging  \
               count      mean       std  min  25%  50%  75%  max       count   
0              389.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0       389.0   
1              444.0  0.013514  0.115590  0.0  0.0  0.0  0.0  1.0       444.0   
2           

In [15]:
fin_df.groupby(['source_file', 'Type']).describe().reset_index().reset_index().dropna()

index  source_file                                       Type  \
                                                                    
0       0      Austria                          Bundesministerium   
1       1      Austria                        district of Austria   
2       2      Austria                   federal state of Austria   
3       3      Austria                    municipality of Austria   
4       4      Belgium                        Belgian delegations   
6       6      Belgium                     Federal Public Service   
7       7      Belgium               Federal Scientific Institute   
8       8      Belgium      Public Institution of Social Security   
9       9      Belgium              Public Interest Organizations   
10     10      Belgium                    Public Planning Service   
16     16      Belgium                                    embassy   
17     17      Belgium                             emergency zone   
20     20      Belgium                    municipality of Belgium   
21     21      Belgium                                police zone   
22     22      Belgium                        province of Belgium   
30     30     Cameroon                       ministry of Cameroon   
44     44   East Timor                    ministry of Timor-Leste   
60     60        Ghana                          Ministry of Ghana   
61     61        Ghana                          district of Ghana   
62     62        Ghana                            region of Ghana   
66     66      Iceland                    municipality of Iceland   
77     77       Latvia                         Ministry of Latvia   
80     80       Latvia                     municipality of Latvia   
85     85     Malaysia                       district of Malaysia   
86     86     Malaysia              federal territory of Malaysia   
87     87     Malaysia                       ministry of Malaysia   
88     88     Malaysia                          state of Malaysia   
97     97        Nepal                          district of Nepal   
98     98        Nepal                          ministry of Nepal   
99     99        Nepal                          province of Nepal   
138   138       Russia                 federal ministry in Russia   
152   152       Sweden                            AP pension fund   
156   156       Sweden                  Swedish government agency   
157   157       Sweden  administrative courts of appeal in Sweden   
158   158       Sweden                authority under the Riksdag   
159   159       Sweden                            court of appeal   
161   161       Sweden                             district court   
164   164       Sweden                     municipality of Sweden   
165   165       Sweden                             public utility   
166   166       Sweden                 regional council in Sweden   
169   169  Switzerland                      canton of Switzerland   
170   170  Switzerland          federal department of Switzerland   
171   171  Switzerland                municipality of Switzerland   
172   172      Ukraine                         Ukrainian ministry   
174   174      Ukraine                          oblast of Ukraine   

    ddg_join_ads                                                 \
           count      mean       std  min   25%  50%   75%  max   
0            8.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0   
1            2.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0   
2            7.0  0.428571  1.133893  0.0  0.00  0.0  0.00  3.0   
3          372.0  0.153226  0.530133  0.0  0.00  0.0  0.00  4.0   
4            6.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0   
6            4.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0   
7            6.0  1.833333  2.041241  0.0  0.00  1.5  3.75  4.0   
8            5.0  0.200000  0.447214  0.0  0.00  0.0  0.00  1.0   
9            7.0  1.571429  1.812654  0.0  0.50  1.0  2.00  5.0   
10           2.0  0.500